# Calculate FRiP Scores for CTCF ChIP-seq Samples

This notebook loops over CTCF BAM replicates, uses **pysam** to count total vs. mapped reads,  
previews a few alignments, then uses **pybedtools** to count reads in MACS3 peaks and compute FRiP  
(Fraction of Reads in Peaks). Finally it collects everything into a **pandas** DataFrame.


In [5]:
#!/usr/bin/env python3
# ─── Cell 1: Imports ───────────────────────────────────────────────────
# - pysam: read & iterate BAM alignments
# - pybedtools: load peaks and intersect with reads
# - pandas: build summary table
import pysam
import pybedtools
import pandas as pd


# ─── Cell 2: Sample & Path Configuration ────────────────────────────────
samples = ["t1", "t2", "t3", "t4"]

# Directory containing CTCF BAM files named CTCF_t1.bam, etc.
bam_dir  = "/projectnb/perissilab/Xinyu/GPS2_CHIPseq/CTCF_3T3L1/results/aligned/"

# Directory containing MACS3 peak calls named CTCF_t1_peaks.narrowPeak, etc.
peak_dir = "/projectnb/perissilab/Xinyu/GPS2_CHIPseq/CTCF_3T3L1/results/macs3/"


# ─── Cell 3: Prepare Results Container ─────────────────────────────────
results = []  # will hold a dict per sample


# ─── Cell 4: Loop over samples ─────────────────────────────────────────
for s in samples:
    bam_file  = f"{bam_dir}CTCF_{s}.bam"
    peak_file = f"{peak_dir}CTCF_{s}_peaks.narrowPeak"
    
    print(f"\n======== Sample {s} ========")
    
    # 1) Summarize BAM: total vs. mapped reads, plus a few read examples
    bam = pysam.AlignmentFile(bam_file, "rb")
    total_reads  = 0
    mapped_reads = 0
    for read in bam.fetch(until_eof=True):
        total_reads += 1
        if not read.is_unmapped:
            mapped_reads += 1
        if total_reads <= 3:
            # preview first 3 alignments
            print(f"{read.reference_name}:{read.reference_start}-{read.reference_end} flag={read.flag}")
    bam.close()
    print(f"Total reads: {total_reads}, Mapped: {mapped_reads} ({mapped_reads/total_reads:.4f})")
    
    # 2) Summarize peaks
    peaks = pybedtools.BedTool(peak_file)
    preview = [f"{p.chrom}:{p.start}-{p.end}" for i,p in enumerate(peaks) if i<3]
    print(f"Peaks preview: {preview}")
    total_peaks = len(peaks)
    print(f"Total peaks: {total_peaks}")
    
    # 3) Compute FRiP: number of reads overlapping any peak / mapped_reads
    bam_bed        = pybedtools.BedTool(bam_file)
    reads_in_peaks = bam_bed.intersect(peaks, u=True).count()
    frip           = reads_in_peaks / mapped_reads if mapped_reads > 0 else 0
    
    # 4) Append metrics
    results.append({
        "Sample":       s,
        "Total Reads":  total_reads,
        "Mapped Reads": mapped_reads,
        "Reads in Peaks": reads_in_peaks,
        "Mapping Rate": round(mapped_reads/total_reads, 4),
        "FRiP":         round(frip, 4)
    })


# ─── Cell 5: Build & Print Summary Table ────────────────────────────────
df = pd.DataFrame(results)
print("\n==== Summary Table ====")
print(df.to_string(index=False))



======== t1 ========
None:-1-None flag=4
None:-1-None flag=4
None:-1-None flag=4
Total: 1493896, Mapped: 1301721 (0.8714)
Peaks preview: ['GL456210.1:62013-62169', 'GL456210.1:98091-98436', 'GL456211.1:20013-20272']
Total peaks: 35467


***** WARNING: File /projectnb/perissilab/Xinyu/GPS2_CHIPseq/CTCF_3T3L1/results/macs3/CTCF_t1_peaks.narrowPeak has inconsistent naming convention for record:
chr1	3051706	3052078	CTCF_t1_peak_278	378	.	18.7645	42.043	37.8865	189

***** WARNING: File /projectnb/perissilab/Xinyu/GPS2_CHIPseq/CTCF_3T3L1/results/macs3/CTCF_t1_peaks.narrowPeak has inconsistent naming convention for record:
chr1	3051706	3052078	CTCF_t1_peak_278	378	.	18.7645	42.043	37.8865	189




======== t2 ========
chr10:83447661-83447697 flag=0
chr11:91412886-91412922 flag=16
chr2:98497461-98497497 flag=0
Total: 1549987, Mapped: 1447573 (0.9339)
Peaks preview: ['GL456210.1:98075-98486', 'GL456210.1:113627-113777', 'GL456211.1:20056-20314']
Total peaks: 30523


***** WARNING: File /projectnb/perissilab/Xinyu/GPS2_CHIPseq/CTCF_3T3L1/results/macs3/CTCF_t2_peaks.narrowPeak has inconsistent naming convention for record:
chr1	3051712	3052051	CTCF_t2_peak_280	494	.	23.3162	53.7552	49.4141	171

***** WARNING: File /projectnb/perissilab/Xinyu/GPS2_CHIPseq/CTCF_3T3L1/results/macs3/CTCF_t2_peaks.narrowPeak has inconsistent naming convention for record:
chr1	3051712	3052051	CTCF_t2_peak_280	494	.	23.3162	53.7552	49.4141	171




======== t3 ========
chr2:98494272-98494308 flag=16
chr6:58748000-58748036 flag=16
None:-1-None flag=4
Total: 2029952, Mapped: 1837371 (0.9051)
Peaks preview: ['GL456210.1:98101-98347', 'GL456211.1:20047-20287', 'GL456211.1:141219-141426']
Total peaks: 46220


***** WARNING: File /projectnb/perissilab/Xinyu/GPS2_CHIPseq/CTCF_3T3L1/results/macs3/CTCF_t3_peaks.narrowPeak has inconsistent naming convention for record:
chr1	3051809	3052057	CTCF_t3_peak_282	648	.	28.8559	68.9452	64.8027	124

***** WARNING: File /projectnb/perissilab/Xinyu/GPS2_CHIPseq/CTCF_3T3L1/results/macs3/CTCF_t3_peaks.narrowPeak has inconsistent naming convention for record:
chr1	3051809	3052057	CTCF_t3_peak_282	648	.	28.8559	68.9452	64.8027	124




======== t4 ========
chr15:12371439-12371475 flag=16
None:-1-None flag=4
None:-1-None flag=4
Total: 845766, Mapped: 740310 (0.8753)
Peaks preview: ['GL456210.1:98098-98340', 'GL456211.1:20043-20303', 'GL456211.1:147822-148074']
Total peaks: 25225


***** WARNING: File /projectnb/perissilab/Xinyu/GPS2_CHIPseq/CTCF_3T3L1/results/macs3/CTCF_t4_peaks.narrowPeak has inconsistent naming convention for record:
chr1	3051777	3052031	CTCF_t4_peak_222	300	.	14.5138	34.2543	30.078	135

***** WARNING: File /projectnb/perissilab/Xinyu/GPS2_CHIPseq/CTCF_3T3L1/results/macs3/CTCF_t4_peaks.narrowPeak has inconsistent naming convention for record:
chr1	3051777	3052031	CTCF_t4_peak_222	300	.	14.5138	34.2543	30.078	135




==== Summary Table ====
Sample  Total Reads  Mapped Reads  Reads in Peaks  Mapping Rate   FRiP
    t1      1493896       1301721          466873        0.8714 0.3587
    t2      1549987       1447573          435556        0.9339 0.3009
    t3      2029952       1837371          744686        0.9051 0.4053
    t4       845766        740310          266471        0.8753 0.3599
